In [ ]:
import os
import logging
from telegram.ext import Updater, MessageHandler, Filters
import IPython.display as ipd

import librosa
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import io


In [ ]:
with open("Telegram-bot-token.txt", "r") as f:
    TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists

HF_MODEL = "cantillation/whisper-tiny-he-teamim-allNusah-14-03-24-warmup-100-RandomFalse"
model = WhisperForConditionalGeneration.from_pretrained(HF_MODEL).to("cuda")
processor = WhisperProcessor.from_pretrained(HF_MODEL, language="hebrew", task="transcribe")
SR = processor.feature_extractor.sampling_rate

In [ ]:
def extract_features(audio):
    feature = processor.feature_extractor(audio, sampling_rate=SR,).input_features[0]
    return torch.tensor(feature).unsqueeze(0)


In [ ]:
def transcribe(audio):

    # preprocess the audio file
    inputs = extract_features(audio).to("cuda")
    
    # generate the text
    generated_ids = model.generate(inputs, max_length=225, num_beams=4, early_stopping=True, return_dict_in_generate=True)
    print(generated_ids.keys())
    # return_dict_in_generate=True so we need to access the "sequences" key
    generated_ids = generated_ids.sequences
    transcription = processor.decode(generated_ids[0], skip_special_tokens=False)
    
    return transcription


In [ ]:
# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

# Define a function to handle audio messages
def handle_audio(update, context):
    audio_message = update.message.voice or update.message.audio
    # Get the audio file
    file = context.bot.get_file(audio_message.file_id)
    audio = file.download_as_bytearray()
    audio = librosa.load(io.BytesIO(audio), sr=SR, mono=True)[0]
    
    # Send a message to the user
    context.bot.send_message(chat_id=update.message.chat_id, text="קיבלתי את הקובץ, אני מתחיל להמיר אותו לטקסט עם טעמים...")
    
    # Audio to text with cantillations
    transcription = str(transcribe(audio))
    
    # Send the transcription to the user
    context.bot.send_message(chat_id=update.message.chat_id, text=f"זה מוכן!: \n {transcription}")
    
    
    
    
def main():
    # Create an instance of the Updater class
    updater = Updater(TOKEN, use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Register a handler for audio messages
    audio_handler = MessageHandler(Filters.audio, handle_audio)
    dispatcher.add_handler(audio_handler)
    
    # Register a handler for voice messages
    voice_handler = MessageHandler(Filters.voice, handle_audio)
    dispatcher.add_handler(voice_handler)
    
    # Start the bot
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

In [ ]:
# Example for how to im
from telegram import ReplyKeyboardMarkup
from telegram.ext import CommandHandler, MessageHandler, Filters, ConversationHandler

# Define a function to handle text messages
def handle_text(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="You said: " + update.message.text)

# Define a function to handle /start command
def start(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="Hello, I'm your bot!")

# Define a function to handle /help command
def help(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text="You can send me an audio message and I will transcribe it for you.")

# Define a function to handle a conversation
def start_conversation(update, context):
    reply_keyboard = [['Option 1', 'Option 2'], ['Option 3', 'Option 4']]
    update.message.reply_text('Please choose:', reply_markup=ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True))
    return 1

def continue_conversation(update, context):
    user_choice = update.message.text
    update.message.reply_text('You chose: ' + user_choice)
    return ConversationHandler.END


# Define a function to handle audio messages
def handle_audio(update, context):
    audio_message = update.message.voice or update.message.audio
    # Get the audio file
    file = context.bot.get_file(audio_message.file_id)
    audio = file.download_as_bytearray()
    audio = librosa.load(io.BytesIO(audio), sr=SR, mono=True)[0]
    
    # Send a message to the user
    context.bot.send_message(chat_id=update.message.chat_id, text="קיבלתי את הקובץ, אני מתחיל להמיר אותו לטקסט עם טעמים...")
    
    # Audio to text with cantillations
    transcription = str(transcribe(audio))
    
    # Send the transcription to the user
    context.bot.send_message(chat_id=update.message.chat_id, text=f"זה מוכן!: \n {transcription}")
    
    
def main():
    # Create an instance of the Updater class
    updater = Updater(TOKEN, use_context=True)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Register a handler for text messages
    text_handler = MessageHandler(Filters.text & (~Filters.command), handle_text)
    dispatcher.add_handler(text_handler)

    # Register a handler for /start command
    start_handler = CommandHandler('start', start)
    dispatcher.add_handler(start_handler)

    # Register a handler for /help command
    help_handler = CommandHandler('help', help)
    dispatcher.add_handler(help_handler)

    # Register a handler for audio messages
    audio_handler = MessageHandler(Filters.audio, handle_audio)
    dispatcher.add_handler(audio_handler)

    # Register a handler for voice messages
    voice_handler = MessageHandler(Filters.voice, handle_audio)
    dispatcher.add_handler(voice_handler)

    # Register a conversation handler
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('conversation', start_conversation)],
        states={
            1: [MessageHandler(Filters.text, continue_conversation)],
        },
        fallbacks=[CommandHandler('cancel', lambda update, context: ConversationHandler.END)]
    )
    dispatcher.add_handler(conv_handler)

    # Start the bot
    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()